In [1]:
import os
os.environ.setdefault('AWS_PROFILE', 'ghactivity')

'ghactivity'

In [2]:
!aws s3 rm s3://aigithub/landing/ghactivity/ --recursive

delete: s3://aigithub/landing/ghactivity/2022-06-05-0.json.gz


In [3]:
import boto3

In [4]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')
table.delete_item(Key={'job_id': 'ghactivity_ingest'})
item = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': '1TK3APN8P6628EL6N8IS33ALIFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 08 Jun 2022 15:47:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1TK3APN8P6628EL6N8IS33ALIFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [5]:
from datetime import datetime as dt
from datetime import timedelta as td
import time

In [6]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('jobs')
    job_details = table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [36]:
def get_next_file_name(job_details):
    job_start_time = int(time.mktime(dt.now().timetuple()))
    job_run_bookmark_details = job_details.get('job_run_bookmark_details')
    baseline_days = int(job_details['baseline_days'])
    if job_run_bookmark_details:
        dt_part = job_run_bookmark_details['last_run_file_name'].split('.')[0].split('/')[-1]
        next_file_name = f"{dt.strftime(dt.strptime(dt_part, '%Y-%m-%d-%H') + td(hours=1), '%Y-%m-%d-%-H')}.json.gz"
    else:
        next_file_name = f'{dt.strftime(dt.now().date() - td(days=baseline_days), "%Y-%m-%d")}-0.json.gz'
    return job_start_time, next_file_name

In [11]:
import requests

In [19]:
def upload_file_to_s3(file_name, bucket_name, folder):
   print(f'Getting the {file_name} from gharchive')
   res = requests.get(f'https://data.gharchive.org/{file_name}')

   print(f'Uploading {file_name} to s3 under s3://{bucket_name}/{folder}')
   s3_client = boto3.client('s3')
   upload_res = s3_client.put_object(
      Bucket=bucket_name,
      Key=f'{folder}/{file_name}',
      Body=res.content
   )

   return {
      'last_run_file_name': f's3://{bucket_name}/{folder}/{file_name}',
      'status_code': upload_res['ResponseMetadata']['HTTPStatusCode']
   }


In [20]:
def save_job_run_details(job_details, job_run_details, job_start_time):
    dynamodb = boto3.resource('dynamodb')
    job_run_details_item = {
        'job_id': job_details['job_id'],
        'job_run_time': job_start_time,
        'job_run_bookmark_details': job_run_details,
        'create_ts': int(time.mktime(dt.now().timetuple()))
    }
    job_run_details_table = dynamodb.Table('job_run_details')
    job_run_details_table.put_item(Item=job_run_details_item)
    
    job_details_table = dynamodb.Table('jobs')
    job_details['job_run_bookmark_details'] = job_run_details
    job_details_table.put_item(Item=job_details)

In [21]:
bucket_name = 'aigithub'

In [22]:
folder = 'landing/ghactivity'

In [23]:
job_details = get_job_details('ghactivity_ingest')
job_start_time, next_file = get_next_file_name(job_details)
job_run_details = upload_file_to_s3(next_file, bucket_name, folder)
save_job_run_details(job_details, job_run_details, job_start_time)

Getting the 2022-06-05-0.json.gz from gharchive
Uploading 2022-06-05-0.json.gz to s3 under s3://aigithub/landing/ghactivity


In [24]:
!aws s3 ls s3://aigithub/landing/ghactivity/

2022-06-08 21:24:04   33989636 2022-06-05-0.json.gz


In [25]:
job_details = get_job_details('ghactivity_ingest')

In [26]:
job_details

{'job_description': 'Ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3'),
 'job_run_bookmark_details': {'last_run_file_name': 's3://aigithub/landing/ghactivity/2022-06-05-0.json.gz',
  'status_code': Decimal('200')}}

In [27]:
dynamodb = boto3.resource('dynamodb')

In [28]:
job_run_details_table = dynamodb.Table('job_run_details')

In [30]:
job_run_details_table.scan()

{'Items': [{'job_id': 'ghactivity_ingest',
   'job_run_bookmark_details': {'processed_file_name': '2022-06-03-0.json.gz'},
   'job_run_time': Decimal('1654577174')},
  {'create_ts': Decimal('1654701247'),
   'job_id': 'ghactivity_ingest',
   'job_run_bookmark_details': {'last_run_file_name': 's3://aigithub/landing/ghactivity/2022-06-05-0.json.gz',
    'status_code': Decimal('200')},
   'job_run_time': Decimal('1654700142')},
  {'create_ts': Decimal('1654703922'),
   'job_id': 'ghactivity_ingest',
   'job_run_bookmark_details': {'last_run_file_name': 's3://aigithub/landing/ghactivity/2022-06-05-0.json.gz',
    'status_code': Decimal('200')},
   'job_run_time': Decimal('1654703636')}],
 'Count': 3,
 'ScannedCount': 3,
 'ResponseMetadata': {'RequestId': 'H794VKFSF6GF48NTDJ505H0ESJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 08 Jun 2022 16:03:31 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '687',

In [37]:
job_start_time, next_file = get_next_file_name(job_details)
job_run_details = upload_file_to_s3(next_file, bucket_name, folder)
save_job_run_details(job_details, job_run_details, job_start_time)

Getting the 2022-06-05-1.json.gz from gharchive
Uploading 2022-06-05-1.json.gz to s3 under s3://aigithub/landing/ghactivity


In [38]:
!aws s3 ls s3://aigithub/landing/ghactivity/

2022-06-08 21:24:04   33989636 2022-06-05-0.json.gz
2022-06-08 21:44:13   39078924 2022-06-05-1.json.gz


In [39]:
dynamodb = boto3.resource('dynamodb')
jobs_table = dynamodb.Table('jobs')
jobs_table.scan()

{'Items': [{'job_description': 'Ingest ghactivity data to s3',
   'is_active': 'Y',
   'job_id': 'ghactivity_ingest',
   'baseline_days': Decimal('3'),
   'job_run_bookmark_details': {'last_run_file_name': 's3://aigithub/landing/ghactivity/2022-06-05-1.json.gz',
    'status_code': Decimal('200')}}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': 'M813GKNU5FATNKCLOOE7D10AQBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 08 Jun 2022 16:20:36 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '322',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'M813GKNU5FATNKCLOOE7D10AQBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1472637769'},
  'RetryAttempts': 0}}

In [41]:
job_run_details_table = dynamodb.Table('job_run_details')
job_run_details_table.scan()

{'Items': [{'job_id': 'ghactivity_ingest',
   'job_run_bookmark_details': {'processed_file_name': '2022-06-03-0.json.gz'},
   'job_run_time': Decimal('1654577174')},
  {'create_ts': Decimal('1654701247'),
   'job_id': 'ghactivity_ingest',
   'job_run_bookmark_details': {'last_run_file_name': 's3://aigithub/landing/ghactivity/2022-06-05-0.json.gz',
    'status_code': Decimal('200')},
   'job_run_time': Decimal('1654700142')},
  {'create_ts': Decimal('1654703922'),
   'job_id': 'ghactivity_ingest',
   'job_run_bookmark_details': {'last_run_file_name': 's3://aigithub/landing/ghactivity/2022-06-05-0.json.gz',
    'status_code': Decimal('200')},
   'job_run_time': Decimal('1654703636')},
  {'create_ts': Decimal('1654705123'),
   'job_id': 'ghactivity_ingest',
   'job_run_bookmark_details': {'last_run_file_name': 's3://aigithub/landing/ghactivity/2022-06-05-1.json.gz',
    'status_code': Decimal('200')},
   'job_run_time': Decimal('1654704838')}],
 'Count': 4,
 'ScannedCount': 4,
 'ResponseM

In [40]:
!aws s3 ls s3://aigithub/landing/ghactivity/2022-06-05-1.json.gz

2022-06-08 21:44:13   39078924 2022-06-05-1.json.gz
